In [1]:
import numpy as np
import torch

import models.TextRNN as model
from utils import DataConfig

np.random.seed(3407)
torch.manual_seed(3407)
torch.cuda.manual_seed_all(3407)
torch.backends.cudnn.deterministic = True  # 保证每次结果一样
torch.backends.cudnn.benchmark = False
UNK, PAD = '<UNK>', '<PAD>'  # 未知字，padding符号

In [2]:
data_config = DataConfig('ship_data', 'embedding.npz')
model_config = model.Config()

In [41]:
import torch
import torch.nn as nn
import torch.nn.functional as f


class Model(nn.Module):
    def __init__(self, model_config, data_config):
        super(Model, self).__init__()
        if data_config.embedding_pretrained is not None:
            self.embedding = nn.Embedding.from_pretrained(data_config.embedding_pretrained, freeze=False)
        else:
            self.embedding = nn.Embedding(data_config.n_vocab, data_config.embed, padding_idx=data_config.n_vocab - 1)
        self.lstm = nn.LSTM(data_config.embed, model_config.hidden_size, model_config.num_layers,
                            bidirectional=True, batch_first=True, dropout=model_config.dropout)
        self.fc = nn.Linear(model_config.hidden_size * 2, data_config.num_classes)

    def forward(self, x):
        out = self.embedding(x)  # [batch_size, seq_len, embeding]
        out, _ = self.lstm(out)
        out = self.fc(out[:, -1, :])  # 句子最后时刻的 hidden state
        return out


In [42]:
model = Model(model_config,data_config).to(data_config.device)

In [43]:
model(torch.randint(1, 10, [64, 30]).to(data_config.device)).size()


torch.Size([64, 5])

In [44]:
from torchinfo import summary

summary(model, input_size=(1, 30),dtypes=[torch.long])

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [1, 5]                    --
├─Embedding: 1-1                         [1, 30, 100]              72,700
├─LSTM: 1-2                              [1, 30, 512]              2,310,144
├─Linear: 1-3                            [1, 5]                    2,565
Total params: 2,385,409
Trainable params: 2,385,409
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 69.38
Input size (MB): 0.00
Forward/backward pass size (MB): 0.15
Params size (MB): 9.54
Estimated Total Size (MB): 9.69

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 读取CSV文件
df = pd.read_csv('./ship_data/experiment_data.csv')  # 替换为你的CSV文件路径

# 划分数据集
train_df, temp_df = train_test_split(df, test_size=0.1, random_state=3407)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=3407)

# 保存划分后的数据集为新的CSV文件
train_df.to_csv('train_dataset.csv', index=False)
val_df.to_csv('val_dataset.csv', index=False)
test_df.to_csv('test_dataset.csv', index=False)
